# Assignment
We have a Python script, which takes input data, computes weights and assigns them to respondents. 
As is often the case, the data must be cleaned first. It’s great if you write down how the dataset 
changes after each step of data cleaning. You can use whatever tools or libraries you like, the goal is to use the optimal 
solution. 

## Data wrangling
You are going to work with data stored in `input_data.csv.zip`.


### 1.	describe the data: what values it mostly contains, what is the size of the dataset etc.

Given the size of the file, I can not read the whole file into the memory to check the size. So I read the first 500 rows just to get a rough idea of what the data look like. It has 2501 columns, mainly integers of 0 and 1. Two columns are float numbers, and one is object.


In [10]:
import pandas as pd
import numpy as np
import os
import csv

CS = 20000
PATH = r'C:\Jian temp\GWIndex'
orig_filename = r'input_data_orig.csv'
q3filename = r'q3_column.csv'
new_filename= r'input_data_new.csv'
result_filename = r'result_data.csv'

df = pd.read_csv(os.path.join(PATH, orig_filename), nrows=500)
print(df.shape)
print(df.get_dtype_counts())
print(df.head())


(500, 2501)
float64       2
int64      2498
object        1
dtype: int64
  respondent_index  dxllm  cmeyg  savjf  xxjbt  joprf  unycm  gyykp  ruqoz  \
0       gwiindex-0      1      0      0      0      0      1      1      1   
1       gwiindex-1      1      1      0      0      1      1      0      0   
2       gwiindex-2      1      1      0      1      0      1      1      0   
3       gwiindex-3      1      0      1      0      0      1      0      1   
4       gwiindex-4      1      0      1      1      1      1      1      0   

   rlknj  ...    hivqb  rvjcd  olrgb  rodxr  jsrni  uxiku  uwcpb  nhxrx  \
0      0  ...        0      1      0      1      0      1      0      0   
1      0  ...        0      1      1      0      1      0      0      1   
2      0  ...        0      1      1      0      0      1      0      0   
3      0  ...        1      1      1      0      0      1      1      1   
4      1  ...        0      0      1      0      1      0      0      1   

   mmvw

### 2.	drop all duplicate respondents based on `respondent_index` column (keep the first occurrence of the respondent)

Since the file is too big to read into the memory, I cannot use pandas DataFrame.drop_duplicates([‘respondent_index’], keep=’first’, inplace=True) to remove the duplicated entries. But if I read the file into chunks, I can only remove duplicates within the chunksize such as 20,000 rows. So finally, I read the whole file in as a csv file line by line, check whether there are duplicate lines, then write out lines that are unique.


In [7]:
with open(os.path.join(PATH, orig_filename), 'r') as in_file, open(os.path.join(PATH, new_filename), 'w') as out_file:
    seen = set()
    for line in in_file:
        if line in seen:
            continue
        seen.add(line)
        out_file.write(line)       
        

### 3.	We will compute weighting based on the `q2` (gender) and `q4` (age group) column. Drop all respondents having NaNs in these columns. Do you know why this could happen and what the possible solutions are?

Use pandas DataFrame[‘q2’].isnull().sum() function to check whether there is any NaNs in that column, the answer is 50, and 51 NaNs in age group. It could be that particular age group has got no response. Possible solution is to go back to check the survey design to see whether do we need to modify that age group value or should we just exclude this age group or replace the values with mean value or other values.

As said in previous section, the data has to be read in chuncks due to the memory limit, so I will define a function to read the whole data and another function to process the data by chunks and finally ouput the result into a csv file.


In [8]:
def read_data(path, filename, cs):
    yield pd.read_csv(os.path.join(path, new_filename), chunksize=cs)
    
q3col = pd.read_csv(os.path.join(PATH, q3filename)) 
q3col.columns = ['respondent_index', 'q3']
print(q3col.shape)

def process_datachunks(data, q3data):
    num = 0        
    for chunks in data:
        for chunk in chunks:
            if chunk['q2'].isnull().sum() > 0:
                chunk.dropna(subset=['q2'], inplace=True)
            if chunk['q4'].isnull().sum() > 0:
                chunk.dropna(subset=['q4'], inplace=True)
            new_data = pd.merge(chunk, q3data, on='respondent_index')
            print(len(new_data.index), '****** new data')
            if len(new_data.index) >= 1 and 0 == num:
                new_data.to_csv(os.path.join(PATH, result_filename), index=False)
                num +=1
            elif len(new_data.index) >= 1 and num > 0:
                new_data.to_csv(os.path.join(PATH, result_filename), mode='a', header=False, index=False)
                
chunks = read_data(PATH, new_filename, CS)
process_datachunks(chunks, q3col)                

(19499, 2)
9499 ****** new data
10000 ****** new data
0 ****** new data
0 ****** new data
0 ****** new data
0 ****** new data


### 4.	There is a `q3_column.csv` file in the directory. Download it and merge/join it on the index to the original dataframe. Are there more variants of (a database) merge/joins? 

After joining the ‘q3_column.csv ’to the original dataframe, the shape of the dataframe downsize to (19499, 2502), that is, 19499 rows and 2502 columns.
